# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

from math import *
from math import sin, cos, sqrt, atan2, radians
import numpy as np
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# any general notebook setup, like log formatting

In [ ]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def calculate_distance(from_coord, to_coord):
    x0 = from_coord[0]
    y0 = from_coord[1]
    x1 = to_coord[0]
    y1 = to_coord[1]

    R = 6373.0

    lat0 = radians(y0)
    lon0 = radians(x0)
    lat1 = radians(y1)
    lon1 = radians(x1)

    dlon = lon1 - lon0
    dlat = lat1 - lat0

    a = sin(dlat / 2)**2 + cos(lat0) * cos(lat1) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
def add_distance_column(df):
    from_coord = ["pickup_longitude","pickup_latitude"]
    to_coord = ["dropoff_longitude","dropoff_latitude"]
    df['distance'] = np.nan
    for index, row in df.iterrows():
        df.loc[index,'distance'] = calculate_distance([row["pickup_longitude"],row["pickup_latitude"]],[row["dropoff_longitude"],row["dropoff_latitude"]])                                       
    return df

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [7]:
import bs4
import requests
def get_taxi_html(TAXI_URL):
    response = requests.get(TAXI_URL)
    html = response.content
    return html

In [ ]:
def find_taxi_parquet_links(TAXI_URL):
    soup=bs4.BeautifulSoup(get_taxi_html(TAXI_URL), 'html.parser')
    res=[]
    par=soup.find_all('a')
    for i in par:
        if 'Yellow Taxi Trip Records' in i:
            res.append(i.get('href'))
    return res

In [ ]:
import re
def find_taxi_csv_urls(TAXI_URL):
    return find_taxi_parquet_links(TAXI_URL)

In [ ]:
def get_and_clean_month_taxi_data(data_path):
    return pd.read_parquet(data_path, engine='pyarrow')

In [ ]:
import os.path
def get_and_clean_taxi_data(TAXI_URL):
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls(TAXI_URL)
    for csv_url in all_csv_urls:
        file_name = re.search('https://d37ci6vzurychx.cloudfront.net/trip-data/(.*).parquet', csv_url).group(1)
        if  int(file_name[-7:-3])<2015 or (int(file_name[-7:-3])==2015 and int(file_name[-2:])<=6):
            if os.path.isfile('./'+file_name+'.parquet'):
            # maybe: first try to see if you've downloaded this exact
            # file already and saved it before trying again
                print(file_name," saved")
                dataframe = pd.read_parquet(file_name+'.parquet')
            else:
                print(file_name," downloading")
                dataframe = get_and_clean_month_taxi_data(csv_url)
                dataframe = dataframe.sample(n=3000, random_state=1)
                dataframe.to_parquet(file_name+'.parquet')
                
            # add_distance_column(dataframe)
            
            # maybe: if the file hasn't been saved, save it so you can
            # avoid re-downloading it if you re-run the function

            all_taxi_dataframes.append(dataframe)
    return pd.concat(all_taxi_dataframes)

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [10]:
def load_and_clean_uber_data(csv_file):
    data = pd.read_csv(csv_file)
    # remove trips outside of the constraint
    data = data[(data['pickup_longitude']>=-74.242330) & (data['pickup_longitude']<=-73.717047) & 
                (data['pickup_latitude']>=40.560445) & (data['pickup_latitude']<=40.908524) &
                (data['dropoff_longitude']>=-74.242330) & (data['dropoff_longitude']<=-73.717047) & 
                (data['dropoff_latitude']>=40.560445) & (data['dropoff_latitude']<=40.908524)]
    # select useful columns
    data = data[['fare_amount','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count']]
    # change to the correct datatype
    data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
    
    return data

In [ ]:
def get_uber_data(UBER_DATA):
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [11]:
def clean_month_weather_data_hourly(csv_file):
    all_df = pd.read_csv(csv_file+'.csv')
    # select useful columns
    all_df = all_df[['STATION','DATE','LATITUDE','LONGITUDE','NAME','HourlyWindSpeed','HourlyPrecipitation']]
    # change to the correct datatype
    all_df['DATE'] = pd.to_datetime(all_df['DATE'], format='%Y-%m-%d %H:%M:%S')
    # deal with missing data
    all_df['HourlyPrecipitation'].fillna(0,inplace=True)
    mask = all_df['HourlyPrecipitation'] == "T"
    all_df.loc[mask, 'HourlyPrecipitation'] = 0
    all_df['HourlyPrecipitation'] = all_df['HourlyPrecipitation'].map(lambda x: str(x).rstrip('s'))
    all_df['HourlyPrecipitation'] = all_df['HourlyPrecipitation'].astype("float64")
    # add new columns for classification
    all_df['day'] = all_df['DATE'].dt.date
    all_df['hour'] = all_df['DATE'].dt.hour
    # classification grouped by hour and date
    all_df = all_df.groupby([all_df['day'],all_df['hour']]).agg({'HourlyWindSpeed':'mean', 'HourlyPrecipitation':'sum'})
    return all_df

In [ ]:
def clean_month_weather_data_daily(csv_file):
    all_df = pd.read_csv(csv_file+'.csv')
    # select useful columns
    all_df = all_df[['STATION','DATE','LATITUDE','LONGITUDE','NAME','HourlyWindSpeed','HourlyPrecipitation']]
    # change to the correct datatype
    all_df['DATE'] = pd.to_datetime(all_df['DATE'], format='%Y-%m-%d %H:%M:%S')
    # deal with missing data
    all_df['HourlyPrecipitation'].fillna(0,inplace=True)
    mask = all_df['HourlyPrecipitation'] == "T"
    all_df.loc[mask, 'HourlyPrecipitation'] = 0
    all_df['HourlyPrecipitation'] = all_df['HourlyPrecipitation'].map(lambda x: str(x).rstrip('s'))
    all_df['HourlyPrecipitation'] = all_df['HourlyPrecipitation'].astype("float64")
    # add new columns for classification
    all_df['day'] = all_df['DATE'].dt.date
    # classification grouped by hour and date
    all_df = all_df.groupby([all_df['day']]).agg({'HourlyWindSpeed':'mean', 'HourlyPrecipitation':'sum'})
    return all_df

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ['2009_weather','2010_weather','2011_weather','2012_weather','2013_weather','2014_weather','2015_weather']
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data(TAXI_URL)
uber_data = get_uber_data(UBER_CSV)
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [14]:
engine = db.create_engine(DATABASE_URL)

In [15]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS people (
   peopleId INTEGER PRIMARY KEY AUTOINCREMENT,
   gn TEXT,
   sn TEXT,
   birthday DATE
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS peopleLog (
   peopleLogId INTEGER PRIMARY KEY AUTOINCREMENT,
   peopleIdNew INTEGER,
   peopleIdOld INTEGER,
   gnNew TEXT,
   gnOld TEXT,
   snNew TEXT,
   snOld TEXT,
   birthdayNew DATE,
   birthdayOld DATE,
   action TEXT,
   timestamp DATE
);

"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS books (
   bookId INTEGER PRIMARY KEY AUTOINCREMENT,
   title TEXT,
   author INTEGER
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS booksLog (
   booksLogId INTEGER PRIMARY KEY AUTOINCREMENT,
   bookIdNew INTEGER,
   bookIdOld INTEGER,
   titleNew TEXT,
   titleOld TEXT,
   authorOld INTEGER);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [16]:
# create the tables with the schema files
with engine.connect() as connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
    connection.execute(DAILY_WEATHER_SCHEMA)
    connection.execute(TAXI_TRIPS_SCHEMA)
    connection.execute(UBER_TRIPS_SCHEMA)

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [17]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [18]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)